In [ ]:
!pip install torch torchvision pycocotools
!pip install torch_xla

In [ ]:
import os
import torch
from torch.utils.data import Dataset
from torchvision.io import read_image
from pycocotools.coco import COCO

class COCODataset(Dataset):
    def __init__(self, root, annotation_file, transforms=None):
        self.root = root
        self.coco = COCO(annotation_file)
        self.ids = list(self.coco.imgs.keys())
        self.transforms = transforms

    def __getitem__(self, index):
        img_id = self.ids[index]
        img_info = self.coco.loadImgs(img_id)[0]
        img_path = os.path.join(self.root, img_info['file_name'])
        image = read_image(img_path).float() / 255.0  # Normalize to [0,1]

        ann_ids = self.coco.getAnnIds(imgIds=img_id)
        anns = self.coco.loadAnns(ann_ids)
        boxes = []
        labels = []
        for ann in anns:
            bbox = ann['bbox']  # [x_min, y_min, width, height]
            boxes.append([bbox[0], bbox[1], bbox[0] + bbox[2], bbox[1] + bbox[3]])
            labels.append(ann['category_id'])

        boxes = torch.tensor(boxes, dtype=torch.float32)
        labels = torch.tensor(labels, dtype=torch.int64)
        target = {"boxes": boxes, "labels": labels}

        if self.transforms:
            image, target = self.transforms(image, target)

        return image, target

    def __len__(self):
        return len(self.ids)


In [ ]:
from torch.utils.data import DataLoader
from pycocotools.coco import COCO
import os

# Paths to images and annotation files
root_dir = "/content/drive/MyDrive/Colab Notebooks/HRSID_JPG/JPEGImages/"
annotation_file = "/content/drive/MyDrive/Colab Notebooks/HRSID_JPG/annotations/train2017.json"

# Load the COCO annotations
coco = COCO(annotation_file)

# Filter valid images
valid_images = []
valid_annotations = []

for img_id in coco.imgs:
    img_info = coco.loadImgs(img_id)[0]
    file_path = os.path.join(root_dir, img_info['file_name'])
    if os.path.exists(file_path):
        valid_images.append(img_info)
        valid_annotations.extend([ann for ann in coco.anns.values() if ann['image_id'] == img_info['id']])
    else:
        print(f"Skipping missing file: {img_info['file_name']}")

# Create a new annotation dictionary
coco_data_filtered = {
    "images": valid_images,
    "annotations": valid_annotations,
    "categories": coco.dataset['categories']
}

# Save the filtered dataset
filtered_annotation_file = "/content/drive/MyDrive/Colab Notebooks/HRSID_JPG/annotations_filtered.json"
with open(filtered_annotation_file, 'w') as f:
    import json
    json.dump(coco_data_filtered, f)

print("Filtered annotations saved.")

filtered_annotation_file = "/content/drive/MyDrive/Colab Notebooks/HRSID_JPG/annotations_filtered.json"
dataset = COCODataset(root=root_dir, annotation_file=filtered_annotation_file)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))


In [ ]:
## Uncomment this section for training (the model is saved after one training)

#import torch
#from torch.optim import Adam
#
#device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
#
#print(f"Using device: {device}")
#model.to(device)
#
## Optimizer
#optimizer = Adam(model.parameters(), lr=0.001)
#
## Training loop
#num_epochs = 10
#for epoch in range(num_epochs):
#    model.train()
#    epoch_loss = 0.0  # Track total loss for the epoch
#
#    for batch_idx, (images, targets) in enumerate(dataloader):
#        images = [image.to(device) for image in images]
#        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
#
#        loss_dict = model(images, targets)
#        losses = sum(loss for loss in loss_dict.values())
#
#        optimizer.zero_grad()
#        losses.backward()
#        optimizer.step()
#
#        epoch_loss += losses.item()
#
#    print(f"Epoch {epoch + 1}, Loss: {epoch_loss:.4f}")

In [ ]:
# Load the model after it's saved
!ls /content
import torch
import torchvision
from torchvision.models.detection import fasterrcnn_resnet50_fpn

# Define the device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Recreate the model
# Load a pretrained Faster R-CNN model
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

# Replace the classifier head with the number of categories
num_classes = len(dataset.coco.getCatIds()) + 1  # Add 1 for the background class
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

# Load the saved weights
model.load_state_dict(torch.load("/content/drive/MyDrive/faster_rcnn_coco.pth", map_location=device))

# Set the model to evaluation mode
model.to(device)
model.eval()

In [ ]:
import torch

# Set model to evaluation mode
model.eval()

# Load a batch of images from the dataloader
images, targets = next(iter(dataloader))
images = [img.to(device) for img in images]

with torch.no_grad():
    predictions = model(images)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Define a function to visualize the predictions
def visualize_predictions(image, prediction, threshold=0.5):
    # Convert image tensor to numpy
    image = image.permute(1, 2, 0).cpu().numpy()

    # Create the plot
    fig, ax = plt.subplots(1, figsize=(12, 8))
    ax.imshow(image)

    # Draw bounding boxes
    for box, label, score in zip(prediction['boxes'], prediction['labels'], prediction['scores']):
        if score >= threshold:
            x_min, y_min, x_max, y_max = box.cpu().numpy()
            width, height = x_max - x_min, y_max - y_min

            # Create a rectangle
            rect = patches.Rectangle((x_min, y_min), width, height, linewidth=2, edgecolor='r', facecolor='none')
            ax.add_patch(rect)

            # Add a confidence score
            ax.text(x_min, y_min - 5, f'{score:.2f}', color='red', fontsize=12, weight='bold')

    plt.show()

# Visualize the first image and its prediction
visualize_predictions(images[3], predictions[3])


In [ ]:
!pip install rasterio
import torch
import rasterio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import torchvision.transforms as transforms

image_path = "/content/drive/MyDrive/pp_20241030T091501_ICEYE_GRD_SC_244270_20220207T192226.tif"

# Load the image
with rasterio.open(image_path) as src:
    img = src.read(1)  # Read the first band (grayscale)
    print(f"Image shape: {img.shape}, dtype: {img.dtype}")

# Apply gamma correction
gamma = 1.5
img = np.clip(img / 255.0, 0, 1)  # Normalize
img = np.power(img, gamma) * 255  # Gamma correction

# Normalize image for model input
min_val, max_val = np.min(img), np.max(img)
normalized_img = (img - min_val) / (max_val - min_val)

# Convert to tensor
image_tensor = torch.tensor(normalized_img, dtype=torch.float32).unsqueeze(0).unsqueeze(0)

# Move to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
image_tensor = image_tensor.to(device)

model.eval()
with torch.no_grad():
    prediction = model(image_tensor)[0]

# Convert grayscale image to 3-channel format for visualization
image_tensor_vis = image_tensor.repeat(1, 3, 1, 1).squeeze(0)  # (1, 1, H, W) → (3, H, W)
visualize_predictions(image_tensor_vis, prediction, threshold=0.5)
